In [15]:
import os
import re
import json
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)
import tensorflow_io as tfio
print(tfio.__version__)

from tensorflow.keras import layers
from tensorflow.keras import models
import tflite_runtime.interpreter as tflite

2.11.0
0.31.0


In [4]:
root_dir = "../data/"
data_path = root_dir + "train.csv"
model_path = "../models/baseline-2Z27DBCM"

In [10]:
df = pd.read_csv(data_path)

def add_path(row):
    return root_dir + row.path

# Get labels 2 id
with open(root_dir+"sign_to_prediction_index_map.json") as f:
    label2id = json.load(f)

df["path"] = df.apply(lambda row: add_path(row), axis=1)
df["sign_encoded"] = df["sign"].apply(lambda sign: label2id[sign])

df.head()

,path,participant_id,sequence_id,sign,sign_encoded
0,../data/train_landmark_files/26734/1000035562....,26734,1000035562,blow,25
1,../data/train_landmark_files/28656/1000106739....,28656,1000106739,wait,232
2,../data/train_landmark_files/16069/100015657.p...,16069,100015657,cloud,48
3,../data/train_landmark_files/25571/1000210073....,25571,1000210073,bird,23
4,../data/train_landmark_files/62590/1000240708....,62590,1000240708,owie,164


In [11]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

SAMPLE_FILE = train_df.path.values[100]

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

frames = load_relevant_data_subset(SAMPLE_FILE)
frames.shape

(59, 543, 3)

In [12]:
NUM_FRAMES = 32

class DataPreprocessing(tf.Module):
    def __init__(self, num_frames=NUM_FRAMES, name=None):
        super().__init__(name=name)
        self.num_frames = num_frames
        
    def true_fn(self, frames, n_frames):
        num_left_frames = self.num_frames - n_frames
        left_frames = tf.zeros(shape=(num_left_frames, 543, 3))
        frames = tf.concat([frames, left_frames], 0)

        return frames

    def false_fn(self, frames):
        frames = tf.slice(
            frames,
            begin=[0,0,0],
            size=[self.num_frames, 543, 3]
        )

        return frames
    
    def shape_list(self, tensor):
        """
        Deal with dynamic shape in tensorflow cleanly.
        Args:
            tensor (`tf.Tensor` or `np.ndarray`): The tensor we want the shape of.
        Returns:
            `List[int]`: The shape of the tensor as a list.
        """
        if isinstance(tensor, np.ndarray):
            return list(tensor.shape)

        dynamic = tf.shape(tensor)

        if tensor.shape == tf.TensorShape(None):
            return dynamic

        static = tensor.shape.as_list()

        return [dynamic[i] if s is None else s for i, s in enumerate(static)]

    def __call__(self, frames):
        n_frames, _, _ = self.shape_list(frames)
        
        # nan to num
        frames = tf.where(tf.math.is_nan(frames), 0.0, frames)

        # sample frames
        frames = tf.cond(
            tf.less(n_frames, NUM_FRAMES),
            true_fn = lambda: self.true_fn(frames, n_frames),
            false_fn = lambda: self.false_fn(frames),
        )

        return tf.expand_dims(frames, axis=0)

In [16]:
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 543, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 32, 468, 3)  0           ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 conv_lstm1d (ConvLSTM1D)       (None, 461, 32)      35968       ['tf.__operators__.getitem[0][0]'
                                                                 ]                            

In [17]:
class TFLiteModel(tf.keras.Model):
    """
    TensorFlow Lite model that takes input tensors and applies:
        – a preprocessing model
        – the ASL model 
    """

    def __init__(self, asl_model):
        """
        Initializes the TFLiteModel with the specified feature generation model and main model.
        """
        super(TFLiteModel, self).__init__()

        # Load the feature generation and main models
        self.prep_inputs = DataPreprocessing()
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def call(self, inputs):
        """
        Applies the feature generation model and main model to the input tensors.

        Args:
            inputs: Input tensor with shape [batch_size, 543, 3].

        Returns:
            A dictionary with a single key 'outputs' and corresponding output tensor.
        """
        x = self.prep_inputs(tf.cast(inputs, dtype=tf.float32))
        outputs = self.model(x)[0, :]

        # Return a dictionary with the output tensor
        return {'outputs': outputs}

tflite_keras_model = TFLiteModel(model)
demo_output = tflite_keras_model(load_relevant_data_subset(SAMPLE_FILE))["outputs"]
np.argmax(demo_output.numpy(), axis=-1)

66

In [18]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflite_keras_model)
tflite_model = keras_model_converter.convert()

model_lite_path = "../models/model.tflite"

with open(model_lite_path, 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp5mvlpyty/assets


INFO:tensorflow:Assets written to: /tmp/tmp5mvlpyty/assets
2023-03-10 16:20:25.682551: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-10 16:20:25.682606: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-10 16:20:25.683706: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp5mvlpyty
2023-03-10 16:20:25.729486: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-10 16:20:25.729528: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp5mvlpyty
2023-03-10 16:20:25.873534: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-10 16:20:25.901817: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-10 16:20:26.126216: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

In [20]:
interpreter = tflite.Interpreter(model_lite_path)
found_signatures = list(interpreter.get_signature_list().items())
print(found_signatures)

prediction_fn = interpreter.get_signature_runner("serving_default")

output = prediction_fn(inputs=load_relevant_data_subset(train_df.path[0]))
sign = np.argmax(output["outputs"])

print(sign)

[('serving_default', {'inputs': ['inputs'], 'outputs': ['outputs']})]
25


In [27]:
%%time

for i in tqdm(range(40000)):
    output = prediction_fn(inputs=load_relevant_data_subset(train_df.path[i]))
    sign = np.argmax(output["outputs"])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40000/40000 [57:53<00:00, 11.52it/s]

CPU times: user 2h 1min 55s, sys: 15min 58s, total: 2h 17min 53s
Wall time: 57min 53s
